<div align="center">
  <a href="http://www.sharif.edu/">
    <img src="https://cdn.freebiesupply.com/logos/large/2x/sharif-logo-png-transparent.png" alt="SUT Logo" width="140">
  </a>
  
  # Sharif University of Technology
  ### Electrical Engineering Department

  ## Signals and Systems
  #### *Final Project - Spring 2025*
</div>

---

<div align="center">
  <h1>
    <b>Object Tracker</b>
  </h1>
  <p>
    An object tracking system using YOLO for detection and various algorithms (KCF, CSRT, MOSSE) for tracking.
  </p>
</div>

<br>

| Professor                  |
| :-------------------------: |
| Dr. Mohammad Mehdi Mojahedian |

<br>

| Contributors              |
| :-----------------------: |
| **Amirreza Mousavi** |
| **Mahdi Falahi** |
| **Zahra Miladipour** |


## 1: Preparing The Materials


In [2]:
import cv2
import matplotlib.pyplot as plt
import numpy as np
from ultralytics import YOLO

1.1 : Calculating HOG ( return the hog of the image)
 

In [3]:
def hog (image):
    # w , h = image.shape()
    filter = cv2.HOGDescriptor()
    result = filter.compute(image)
    return result

1.2 : Checking The Scale (return the scale of the image in the current frame)

In [4]:
def scaled_check (image ,h, source_hog):
    w , h = image.shape()
    n = np.arange(0.9 , 1.2 , 0.05)
    source = np.fft(source_hog)
    h_fft = np.fft(h)
    x = h_fft * source
    scale = 1
    min = 500
    for coefficient in n :
        new_img = cv2.resize(image , coefficient * w , coefficient * h)
        new_fft = np.fft(new_img)
        new_result = new_fft * h_fft
        diff = new_result - x
        sum = np.sum(diff ** 2)
        if (sum < min ):
            min = sum
            scale = coefficient 
    return scale    

1.3 : Prediction The Next Frame's Center ( Kalman Filter )

In [5]:
def kalman_prediction( F , X_k_1 , P_k_1 , Q_k):
    x_k = np.dot(F , X_k_1)
    p_k = np.dot( F , np.dot(P_k_1 , F.T)) + Q_k
    return x_k , p_k

In [6]:
def kalman_updating(x_k , p_k , H_k , z_k , R_k):
    k_1 = np.dot(np.dot(H_k , p_k) , H_k.T) + R_k
    k_2 = np.dot(p_k , H_k.T)
    K = np.dot(k_2 , np.linalg.inv(k_1))
    P_k_new = p_k - np.dot(np.dot(K , H_k) , p_k)
    x_k_new = x_k + np.dot(K , (z_k - np.dot(H_k , x_k)))
    return x_k_new , P_k_new

1.4 : Updating Method

In [7]:
def filter_updating(H_new , H_old , alpha):
    result = alpha * H_new + (1-alpha) * H_old
    return result

## 2 : Main Detector

2.1 : Uploading Video

In [10]:
cap = cv2.VideoCapture('person1.mp4')

while True:
    ret, frame = cap.read()
    if ret:
        cv2.imshow("capture", frame)
    else:
        break

    key = cv2.waitKey(4)
    if key == 27 :
        break
        
cap.release()
cv2.destroyAllWindows()

Video has ended or failed to read frame.
